# GUÍA BRAZO ROBÓTICO EN GAZEBO


## Iniciando la simulacion de Gazebo

Para simular brazo robótico UR10 junto con el gripper en Gazebo, debes seguir estos pasos:

1. Abre una Nueva Terminal: Primero, necesitas abrir una nueva ventana de terminal en tu sistema (Ctrl + Alt + T).

2. Ejecuta el Comando de Lanzamiento: En la terminal recién abierta, ejecuta el siguiente comando:


``` shellscript
user@pc:~$ roslaunch manipulacion_pkg  robot_gazebo.launch tipo_gripper:=jaco \
            objeto:=mustard_bottle

```

## Ejemplo: Control de un Brazo Robótico en Gazebo

En este ejemplo, exploraremos cómo leer las posiciones actuales de las articulaciones de un brazo robótico en Gazebo y cómo enviar comandos para mover el brazo a nuevas posiciones. Utilizaremos la clase `GazeboRobot`, que hereda de `Robot`, y proporciona funcionalidades específicas para interactuar con el brazo robótico en la simulación de Gazebo.

1. **Inicializar el Nodo ROS**: Primero, inicializamos un nodo ROS. Este nodo nos facilitará la comunicación con el sistema ROS y permitirá el control del brazo robótico en la simulación de Gazebo.

2. **Creación de una Instancia de GazeboRobot**: Se crea una instancia de `GazeboRobot`, especificando los nombres de las articulaciones del brazo robótico que queremos controlar. Esto nos permite interactuar específicamente con nuestro brazo robótico en Gazebo.

3. **Lectura de Posiciones Actuales de las Articulaciones**: Utilizamos el método `obtener_posiciones_articulaciones` para leer las posiciones actuales de las articulaciones del brazo robótico. 

4. **Envío de Comando a una Posición articular deseada**:  Para mover el brazo robótico a una posición deseada, usamos el método `command_posicion_articulaciones`, especificando las posiciones deseadas para las articulaciones (`posicion_deseada`) y el tiempo desde que se envía el comando hasta alcanzar estas posiciones (`time_from_start`).

5. **Envío de Comando de un path de Posiciones articulares**: Utilizamos el método `command_path_posicion_articulaciones` para definir una trayectoria de posiciones articulares. Aquí, `path` representa una lista de configuraciones articulares que el brazo debe alcanzar de manera secuencial. `time_between_points` indica el tiempo en segundos necesario para que el brazo se mueva de una configuración a la siguiente dentro de la trayectoria. Por último, `start_time` define el tiempo en segundos desde el envío del comando hasta que el brazo llega a la primera posición de la trayectoria.





In [13]:
import rospy
import manipulacion_lib  

# 1. Inicializar el Nodo ROS
if not rospy.get_node_uri():
    rospy.init_node('robot_arm_controller', anonymous=True)

# 2. Creación de una Instancia de `GazeboRobot`
gazebo_robot = manipulacion_lib.GazeboRobot(nombres_articulaciones=[
    'shoulder_pan_joint',
    'shoulder_lift_joint',
    'elbow_joint',
    'wrist_1_joint',
    'wrist_2_joint',
    'wrist_3_joint'
])

# 3. Lectura de Posiciones Actuales de las Articulaciones
posiciones_actuales = gazebo_robot.obtener_posiciones_articulaciones()
print("Posiciones actuales de las articulaciones:", posiciones_actuales)

# 4. Envío de Comando a una Posición Específica
posicion_deseada = [0.5, -1.0, 1.0, -1.0, 0.0, 1.5]
# Define la posición deseada para las articulaciones y el tiempo para alcanzarla
gazebo_robot.command_posicion_articulaciones(posicion_deseada,
                                             time_from_start=2)


Posiciones actuales de las articulaciones: [0.5000448062945635, 0.006101768656558093, 0.0016628430346559142, 0.4964334722785626, 0.0010352478625401673, 1.496450914142093]


Unknown tag "surface" in /robot[@name='ur10']/link[@name='hand_6_hand_limb']
Unknown tag "hardwareInterface" in /robot[@name='ur10']/transmission[@name='hand_finger_joint_0_pos_transmission']/actuator[@name='hand_finger_joint_0_effort_motor']
Unknown tag "hardwareInterface" in /robot[@name='ur10']/transmission[@name='hand_finger_joint_2_pos_transmission']/actuator[@name='hand_finger_joint_2_effort_motor']
Unknown tag "hardwareInterface" in /robot[@name='ur10']/transmission[@name='hand_finger_joint_4_pos_transmission']/actuator[@name='hand_finger_joint_4_effort_motor']


In [14]:
# 5. Envío de Comando path the posiciones articulares
path = [
    [0.0, -0.8, 0.0, -0.3, 0.0, 0.5],
    [0.0, -1.0, 1.0, -1.0, 0.0, 1.0],
    [0.5, -0.5, 0.5, -0.5, 0.6, 0.5]
]
gazebo_robot.command_path_posicion_articulaciones(path, time_between_points=2,
                                                  start_time=2)


header: 
  seq: 0
  stamp: 
    secs: 657
    nsecs: 605000000
  frame_id: ''
joint_names: 
  - shoulder_pan_joint
  - shoulder_lift_joint
  - elbow_joint
  - wrist_1_joint
  - wrist_2_joint
  - wrist_3_joint
points: 
  - 
    positions: [0.0, -0.8, 0.0, -0.3, 0.0, 0.5]
    velocities: []
    accelerations: []
    effort: []
    time_from_start: 
      secs: 2
      nsecs:         0
  - 
    positions: [0.0, -1.0, 1.0, -1.0, 0.0, 1.0]
    velocities: []
    accelerations: []
    effort: []
    time_from_start: 
      secs: 4
      nsecs:         0
  - 
    positions: [0.5, -0.5, 0.5, -0.5, 0.6, 0.5]
    velocities: []
    accelerations: []
    effort: []
    time_from_start: 
      secs: 6
      nsecs:         0


## Detección de colisiones del brazo robótico

El detector de colisiones debe ser capaz de identificar tanto auto-colisiones del brazo robótico (colisiones entre las distintas partes del brazo y el gripper) como colisiones con obstáculos definidos en el entorno. Consideraremos el gripper como un prisma rectangular para fines de detección de colisiones, usando las dimensiones específicas proporcionadas.

1. **Inicializar el Nodo ROS**: Comenzamos inicializando un nodo ROS. Este nodo facilitará la comunicación con otras partes del sistema ROS.

2. **Definición de Obstáculos**: A continuación, definimos los obstáculos presentes en el entorno que podrían interactuar con el brazo robótico. Cada obstáculo se define con un tipo (por ejemplo, 'cubo'), una pose inicial, dimensiones y un nombre único.


3. **Creación de una Instancia del Detector de Colisiones**: Instanciamos el `DetectorColisiones`, especificando si se incluirá el brazo robótico y/o el gripper, junto con los obstáculos definidos. 


4. **Verificación de Auto-colisiones y Colisiones con Obstáculos**: Utilizamos el método `hay_colision` del detector de colisiones para determinar si, dada una configuración específica de las articulaciones del brazo, existe alguna colisión con los obstáculos definidos o auto-colisiones.




In [3]:
import rospy
import manipulacion_lib

# 1. Inicializar el Nodo ROS
# Asegúrate de que el nodo solo se inicialice si aún no existe
if not rospy.get_node_uri():
    rospy.init_node('collision_detection', anonymous=True)

# 2. Definición de Obstáculos
# Crea una lista para almacenar los obstáculos definidos
obstaculos = []

# Crea y añade un obstáculo específico a la lista
# Asume que manipulacion_lib.Obstaculo es una clase que permite definir
# obstáculos con tipo, pose, dimensiones y nombre
obstaculo = manipulacion_lib.Obstaculo('cubo', [0.7, 0, 0, 1, 0, 0, 0], 
                                       [0.8, 0.3, 0.7], 'obstaculo')
obstaculos.append(obstaculo)
obstaculos = []

# 3. Creación de una Instancia del Detector de Colisiones
# Inicializa el detector de colisiones con la configuración deseada, incluyendo 
# el brazo robótico, el gripper y los obstáculos
detectorColisiones = manipulacion_lib.DetectorColisiones(
    usa_brazo_robotico=True,  # Indica si el brazo robótico se incluye en la 
                              # detección de colisiones
    usa_gripper=True,  # Indica si el gripper se incluye 
                       # en la detección de colisiones
    gripper_dimensions=[0.1, 0.1, 0.1],  # Especifica las dimensiones del 
                                        # gripper como un prisma rectangular
    obstaculos=obstaculos  # Lista de obstáculos definidos previamente
)

# 4. Verificación de Auto-colisiones y Colisiones con Obstáculos
# Define una configuración de ejemplo para las articulaciones del brazo robótico
configuracion_articulaciones = [0, 0, 0, 0, 0, 0]

# Utiliza el método hay_colision para verificar si la configuración actual del
# brazo robótico resulta en alguna colisión
hay_colision = detectorColisiones.hay_colision(configuracion_articulaciones)

# Imprime el resultado de la detección de colisiones
print(f"¿Hay colisión?: {'Sí' if hay_colision else 'No'}")

configuracion_articulaciones = [0, 0.05, 0, 0, 0, 0]
# Utiliza el método hay_colision para verificar si la configuración actual del brazo
# robótico resulta en alguna colisión
hay_colision = detectorColisiones.hay_colision(configuracion_articulaciones)

# Imprime el resultado de la detección de colisiones
print(f"¿Hay colisión?: {'Sí' if hay_colision else 'No'}")



usando gripper
[0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.
 0. 0. 1.]
num collision pairs - initial: 43
base_inertia:  1
shoulder:  2
upper:  3
forearm:  4
wrist1:  5
wrist2:  6
wrist3:  7
gripper:  8
suelo:  9
¿Hay colisión?: No
¿Hay colisión?: No


# Transición de FakeRobot a GazeboRobot para Cálculos Cinemáticos

Al pasar de usar `FakeRobot` con RViz a `GazeboRobot` en Gazebo, el proceso para calcular la cinemática del brazo robótico es el mismo. Solo necesitas cambiar la instancia de `FakeRobot` por `GazeboRobot`. El resto del proceso y los métodos utilizados para calcular la cinemática son exactamente iguales.



In [4]:
import manipulacion_lib

# Cambia FakeRobot por GazeboRobot para simulaciones en Gazebo
ur10 = manipulacion_lib.GazeboRobot()

# El uso de la cinemática se mantiene igual
kin = manipulacion_lib.Cinematica(robot=ur10, frame_base='base_link', 
                                  frame_final='tool0')

Unknown tag "surface" in /robot[@name='ur10']/link[@name='hand_6_hand_limb']
Unknown tag "hardwareInterface" in /robot[@name='ur10']/transmission[@name='hand_finger_joint_0_pos_transmission']/actuator[@name='hand_finger_joint_0_effort_motor']
Unknown tag "hardwareInterface" in /robot[@name='ur10']/transmission[@name='hand_finger_joint_2_pos_transmission']/actuator[@name='hand_finger_joint_2_effort_motor']
Unknown tag "hardwareInterface" in /robot[@name='ur10']/transmission[@name='hand_finger_joint_4_pos_transmission']/actuator[@name='hand_finger_joint_4_effort_motor']
Unknown tag "surface" in /robot[@name='ur10']/link[@name='hand_6_hand_limb']
Unknown tag "hardwareInterface" in /robot[@name='ur10']/transmission[@name='hand_finger_joint_0_pos_transmission']/actuator[@name='hand_finger_joint_0_effort_motor']
Unknown tag "hardwareInterface" in /robot[@name='ur10']/transmission[@name='hand_finger_joint_2_pos_transmission']/actuator[@name='hand_finger_joint_2_effort_motor']
Unknown tag "har

# Transición de SimulacionGripperFlotante a SimulacionGripper para abrir y cerrar el gripper 

Al pasar de usar `SimulacionGripperFlotante` con la simulación del gripper flotante en gazebo a `SimulacionGripper` para usar el gripper con el brazo robótico, el proceso para abrir y cerra el gripper es el mismo. Solo necesitas cambiar la instancia de `SimulacionGripperFlotante` por `SimulacionGripper`. 


In [5]:
simulacion_gripper = manipulacion_lib.SimulacionGripper(nombre_gripper_gazebo="gripper")
# Configurar el gripper basado en el archivo YAML específico para su tipo.
# Esto incluye cargar las configuraciones iniciales como nombres de 
# articulaciones y posiciones predeterminadas.
simulacion_gripper.configurar_gripper()

# Abrir completamente el gripper antes de intentar cualquier operación 
# de agarre.
# Es una práctica común asegurarse de que el gripper no esté restringido 
# o en una posición que podría interferir con el objeto a agarrar.
simulacion_gripper.abrir_gripper()

# Pequeña pausa para asegurar que la acción de abrir se ha completado.
rospy.sleep(1)

# Obtener el tipo de gripper actual para aplicar la configuración 
# de posiciones de articulaciones correcta.
tipo_gripper = simulacion_gripper.get_tipo_gripper()

# Basado en el tipo de gripper, se definen las posiciones 
# específicas de las articulaciones.
# Estas posiciones deben ser determinadas experimentalmente o 
# calculadas para lograr el agarre deseado.
if tipo_gripper == 'jaco': # Posiciones para el gripper Jaco.
  posicion_articulaciones = [-0.6,-0.6,-0.6]  
elif tipo_gripper == 'schunk': # Posiciones para el gripper Schunk.
  posicion_articulaciones = [0, 0, 0, 0, 0, 0, 0]  
elif tipo_gripper == 'robotiq':  # Posiciones para el gripper Robotiq.
  posicion_articulaciones = [0.4, 0.4, 0.4, 0, 0.4, 0.4, 0.4, 0, 0.4, 0.4, 0.4] 

# Aplicar las posiciones definidas a las articulaciones del gripper.
# Este paso es crucial para controlar el estado del gripper y realizar agarres 
# efectivos en la simulación.
simulacion_gripper.set_posicion_articulaciones(posicion_articulaciones)

Gripper posicion articulaciones:  [0.1, 0.1, 0.1]
Gripper posicion articulaciones:  [-0.6, -0.6, -0.6]


## Cálculo de la Trayectoria en el Espacio Articular

En esta sección, se presenta cómo calcular una trayectoria en el espacio articular para un brazo robótico en Gazebo, utilizando el algoritmo RRT* para encontrar un camino que evite colisiones y respete los límites articulares. Utilizaremos la clase `GazeboRobot`, específicamente con una configuración para el robot UR10, y la librería `manipulacion_lib` que proporciona herramientas para la planificación de movimientos.

Para configurar el algoritmo RRT*, se inicializa con los siguientes parámetros:

  - *start*: Posición inicial de las articulaciones del robot.

  - *goal*: Posición final deseada de las articulaciones del robot.
  - *joint_limits*: Una lista de tuplas que define los límites mínimos y máximos para cada articulación.
  - *expand_dis*: La distancia máxima entre nodos al expandir el árbol.
  - *max_iter*: El número máximo de iteraciones para la ejecución del algoritmo.
  - *search_radius*: El radio de búsqueda para conectar con otros nodos en el árbol.
  - *collision_detector*: Un objeto `DetectorColisiones` que permite evaluar si una configuración articulada propuesta resulta en una colisión.

### Pasos:

1. **Cargar el Modelo del Robot UR10**: Creamos una instancia de `GazeboRobot` para el robot UR10, lo que nos permite interactuar con este modelo específico en la simulación de Gazebo.

2. **Obtener los Límites de las Articulaciones:** Recuperamos los límites inferiores y superiores de las articulaciones del robot UR10. Esto es crucial para garantizar que las posiciones articulares calculadas se mantengan dentro de rangos operativos seguros.

3. **Configuración y Ejecución del Algoritmo RRT*:** Configuramos el algoritmo RRT* con los parámetros descritos anteriormente, proporcionando la posición actual de las articulaciones, la posición articular deseada, los límites de las articulaciones, y parámetros del algoritmo como el radio de búsqueda y el número máximo de iteraciones. Así como el detector de colisiones.

4. **Planificación de la Trayectoria**: Ejecutamos el método plan() del objeto RRT* para calcular la trayectoria. Si se encuentra un camino viable, este se imprime en la consola.



In [15]:
posiciones_articulares_deseadas =  [0.5, 0.0, 0.0, 0.5, 0.0, 1.5]

ur10 = manipulacion_lib.GazeboRobot() # Cargamos el robot UR10

# Obtener los Límites de las Articulaciones
min_limits = ur10.get_limites_inferiores()
max_limits = ur10.get_limites_superiores()
joint_limits = []
for i in range(len(min_limits)):
    joint_limits.append((min_limits[i], max_limits[i]))

# Configuración del Algoritmo RRT*
rrt = manipulacion_lib.RRTStarJointSpace(ur10.obtener_posiciones_articulaciones(),
                                         posiciones_articulares_deseadas, joint_limits, 0.5, 200,1.2, detectorColisiones)

# Planificación de la Trayectoria
path = rrt.plan()
if path:
    print(path)



Unknown tag "surface" in /robot[@name='ur10']/link[@name='hand_6_hand_limb']
Unknown tag "hardwareInterface" in /robot[@name='ur10']/transmission[@name='hand_finger_joint_0_pos_transmission']/actuator[@name='hand_finger_joint_0_effort_motor']
Unknown tag "hardwareInterface" in /robot[@name='ur10']/transmission[@name='hand_finger_joint_2_pos_transmission']/actuator[@name='hand_finger_joint_2_effort_motor']
Unknown tag "hardwareInterface" in /robot[@name='ur10']/transmission[@name='hand_finger_joint_4_pos_transmission']/actuator[@name='hand_finger_joint_4_effort_motor']
Unknown tag "surface" in /robot[@name='ur10']/link[@name='hand_6_hand_limb']
Unknown tag "hardwareInterface" in /robot[@name='ur10']/transmission[@name='hand_finger_joint_0_pos_transmission']/actuator[@name='hand_finger_joint_0_effort_motor']
Unknown tag "hardwareInterface" in /robot[@name='ur10']/transmission[@name='hand_finger_joint_2_pos_transmission']/actuator[@name='hand_finger_joint_2_effort_motor']
Unknown tag "har

[[0.5000444186318989, -0.49427642077881284, 0.5018146664312226, -0.4977148887890088, 0.599490913489853, 0.5086727396736181], [0.4635018820130166, -0.2516960949578553, 0.39675394233533023, 0.04447704836384128, 0.5761415157446416, 0.9953301192052251], [0.48160464076775406, -0.12685695426143562, 0.19996733252577803, 0.2704125912226877, 0.29038018219221945, 1.245642124500077], [0.5, 0.0, 0.0, 0.5, 0.0, 1.5], [0.5, 0.0, 0.0, 0.5, 0.0, 1.5]]


In [17]:
ur10.command_path_posicion_articulaciones(path, 0.4, 1.0)

header: 
  seq: 0
  stamp: 
    secs: 858
    nsecs:   4000000
  frame_id: ''
joint_names: 
  - shoulder_pan_joint
  - shoulder_lift_joint
  - elbow_joint
  - wrist_1_joint
  - wrist_2_joint
  - wrist_3_joint
points: 
  - 
    positions: [0.5000444186318989, -0.49427642077881284, 0.5018146664312226, -0.4977148887890088, 0.599490913489853, 0.5086727396736181]
    velocities: []
    accelerations: []
    effort: []
    time_from_start: 
      secs: 1
      nsecs:         0
  - 
    positions: [0.4635018820130166, -0.2516960949578553, 0.39675394233533023, 0.04447704836384128, 0.5761415157446416, 0.9953301192052251]
    velocities: []
    accelerations: []
    effort: []
    time_from_start: 
      secs: 1
      nsecs: 400000000
  - 
    positions: [0.48160464076775406, -0.12685695426143562, 0.19996733252577803, 0.2704125912226877, 0.29038018219221945, 1.245642124500077]
    velocities: []
    accelerations: []
    effort: []
    time_from_start: 
      secs: 1
      nsecs: 800000000
  - 
